In [2]:
import numpy as np
import random
from inputdata import Course, Room, Student, Teacher
from Schedule import Schedule 
from MasterSchedule import MasterSchedule
from RoomSchedule import RoomSchedule
from StudentSchedule import StudentSchedule
from TeacherSchedule import TeacherSchedule
from Section import Section
from Constants import DAYS_IN_WEEK, NUMBER_OF_PERIODS

In [3]:
def ChangeSectionPeriod(master_schedule : MasterSchedule):
  room : RoomSchedule = random.choice(master_schedule.roomSchedules)
  orig_period = random.randrange(0, NUMBER_OF_PERIODS)
  new_period
  while new_period == orig_period:
    new_period = random.randrange(0, NUMBER_OF_PERIODS)
  
  for i in range(DAYS_IN_WEEK):
    a = room.sections[i][orig_period]
    b = room.sections[i][new_period]
    room.sections[i][orig_period] = b
    room.sections[i][new_period] = a



In [5]:
def SplitSectionPeriod(master_schedule : MasterSchedule):
  orig_room : RoomSchedule = random.choice(master_schedule.roomSchedules)
  while orig_room.getOccupiedSectionTimes().size() == 0:
    orig_room = random.choice(master_schedule.roomSchedules)
  occupied = orig_room.getOccupiedSectionTimes()

  orig_period = random.choice(occupied)[1]

  sec_room : RoomSchedule = random.choice(master_schedule.roomSchedules)
  while sec_room.getEmptySectionTimes().size() == 0:
    sec_room = random.choice(master_schedule.roomSchedules)
  unoccupied = sec_room.getEmptySectionTimes()

  sec_period = random(unoccupied)[1]

  teacher = orig_room.getScheduledSection(0, orig_period).getTeacher()
  
